In [3]:
from datetime import datetime
from concurrent import futures

import pandas as pd
import numpy as np
from pandas import DataFrame
import pandas_datareader.data as web
from pandas_datareader import data

import time,datetime
import glob
import fix_yahoo_finance as yf
yf.pdr_override()


from informs import portfolio
import class_informs as inform
import generator
import discriminator

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm

import cmath

import re
import matplotlib.pyplot as plt

In [4]:
SP500=pd.read_csv('data/s&p500.csv')
SP100=pd.read_csv('data/s&p100.csv')
SP400=pd.read_csv('data/s&p400.csv', encoding= "ISO-8859-1")
SP400=SP400.rename(columns={'Ticker Symbol' : 'Symbol'})

In [11]:

def get_stockdata(stocks,start_date,end_date):
    
    fail_names =[] 

    df=pd.DataFrame()
    
    for stock in stocks:
        try:
            stock_df = data.get_data_yahoo(stock, start_date, end_date)
            stock_df['Name'] = stock
            df=df.append(stock_df)
        except:
            print('bad: %s' % (stock))
            fail_names=fail_names.append(stock)

            
    return df,fail_names


def bench(df,NEW_stocks):
        
    THIS=df[df.index==df.index[-1]]

    THIS['tot_market']=THIS['Close']*THIS['Volume']
    THIS['bench']=THIS['tot_market']/THIS['tot_market'].sum()

    dic_bench={}
    for i in list(THIS['Name']):
        dic_bench[i]=float(THIS[THIS['Name']==i]['bench'])

    return dic_bench,THIS



def sector_mcapq(THIS):
        
    INFO=pd.read_csv('data/s&p500 list.csv')
    INFO=INFO.rename(columns={'Ticker symbol' : 'Name'})


    THIS=pd.merge(THIS, INFO, how='left')
    THIS=THIS.rename(columns={'GICS_Sector' : 'Sector'})

    THIS['rank'] = THIS['bench'].rank(ascending=0,method='min')
    THIS['mcapq'] = np.where(THIS['rank']<=100, '1', np.where(THIS['rank']<=200, '2', np.where(THIS['rank']<=300, '3', np.where(THIS['rank']<=400, '4', '5'))))

    
    dic_sector = {THIS["Sector"][i] : [] for i in THIS.index }

    dic_MCAP = {THIS["mcapq"][i] : [] for i in THIS.index}

    for i in THIS.index:
        dic_sector[THIS["Sector"][i]].append(THIS["Name"][i])
        dic_MCAP[THIS["mcapq"][i]].append(THIS["Name"][i])
    THIS['index']=list(THIS.index)
    
    return dic_sector,dic_MCAP,THIS
        
        
def omega_covariances(prices):
    prices =np.matrix(prices)
    rows, cols = prices.shape
    returns = np.empty([rows, cols - 1])
    for r in range(rows):
        for c in range(cols - 1):
            p0, p1 = prices[r, c], prices[r, c + 1]
            returns[r, c] = (p1 / p0) - 1
    covars = np.cov(returns)
    covars = covars * len(prices[0]) 
    return covars


def alpha_return(stocks,start_date,end_date):
    
    close = pd.DataFrame()
    fail_names=[]

    for stock in stocks:
        try:
            close[stock] =  data.get_data_yahoo(stock, start_date, end_date)['Adj Close']
        except:
            fail_names.append(stock)

    daily_simple_returns = close.pct_change()
    annual_returns = daily_simple_returns.mean() * len(close)
    
    return close,annual_returns,fail_names



In [12]:
#Data Setting
start_date = '2017-02-16'
end_date = '2018-02-15'
stocks=list(SP500['Symbol'])


#Data Loading
SP500_df,fail_name=get_stockdata(stocks,start_date,end_date)
SP500_df.to_csv('./data/SP500_df.csv')


#Omega
prices=list(SP500_df.groupby('Name')['Close'].apply(list))
date_len=len(SP500_df.index.unique().tolist())
for i in range(len(prices)):
    if len(prices[i]) != date_len:
        prices[i].extend([prices[i][-1]]*(date_len-len(prices[i])))
        
risk_mat = omega_covariances(prices)
NEW_stocks=list(SP500_df.groupby('Name')['Close'].describe().index)
        #risk_sedol->NEW_stocks

#Alpha
close,dic_alpha,fail_names2 = alpha_return(NEW_stocks,start_date,end_date)
        #dic_sedol_as->dic_alpha    
    

#Bench Weight
dic_bench,THIS = bench(SP500_df,NEW_stocks)


#sector, MCAPQ
dic_sector,dic_MCAP,THIS = sector_mcapq(THIS)



/Users/yerin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yerin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
import pickle

with open('all_data.txt', 'wb') as f:
    pickle.dump(risk_mat, f)
    pickle.dump(NEW_stocks, f)
    pickle.dump(close, f)
    pickle.dump(dic_alpha, f)
    pickle.dump(dic_bench, f)
    pickle.dump(dic_sector, f)
    pickle.dump(dic_MCAP, f)
    pickle.dump(THIS, f)
